<a href="https://colab.research.google.com/github/nazzang49/boost-camp-projects/blob/main/assignments/(P09)GPT_fine_tuning_practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국어 언어모델 학습 및 다중 과제 튜닝
## KoGPT-2 기반의 챗봇 실습

> 작성자      
```
* 김성현 (bananaband657@gmail.com)  
1기 멘토
김바다 (qkek983@gmail.com)
박상희 (parksanghee0103@gmail.com)  
이정우 (jungwoo.l2.rs@gmail.com)
2기 멘토
박상희 (parksanghee0103@gmail.com)  
이정우 (jungwoo.l2.rs@gmail.com)
이녕우 (leenw2@gmail.com)
박채훈 (qkrcogns2222@gmail.com)
```
[CC BY-NC-ND](https://creativecommons.org/licenses/by-nc-nd/2.0/kr/)

In [ ]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash
!apt-get install git-lfs

In [2]:
!git lfs install
!git clone https://huggingface.co/taeminlee/kogpt2

Git LFS initialized.
Cloning into 'kogpt2'...
remote: Enumerating objects: 52, done.
remote: Counting objects: 100% (52/52), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 52 (delta 20), reused 0 (delta 0)
Unpacking objects: 100% (52/52), done.
Filtering content: 100% (2/2), 959.93 MiB | 64.77 MiB/s, done.


In [ ]:
!pip install transformers

In [ ]:
import torch
from tokenizers import SentencePieceBPETokenizer
from transformers import GPT2Config, GPT2LMHeadModel

tokenizer = SentencePieceBPETokenizer("/content/kogpt2/vocab.json", "/content/kogpt2/merges.txt")

config = GPT2Config(vocab_size=50000)
model = GPT2LMHeadModel(config)

model_dir = '/content/kogpt2/pytorch_model.bin'

model.load_state_dict(torch.load(model_dir, map_location='cuda'), strict=False)
model.to('cuda')

In [5]:
tokenized_text = tokenizer.encode('이순신은 조선 중기의 무신이다.', add_special_tokens=True)
print(tokenized_text)
print(tokenized_text.tokens)
print(tokenized_text.ids)

Encoding(num_tokens=7, attributes=[ids, type_ids, tokens, offsets, attention_mask, special_tokens_mask, overflowing])
['▁이순', '신은', '▁조선', '▁중기의', '▁무신', '이다', '.']
[10925, 6647, 1117, 40249, 39793, 128, 47440]


In [6]:
import torch
torch.manual_seed(42)

input_ids = torch.tensor(tokenizer.encode("이순신은", add_special_tokens=True).ids).unsqueeze(0).to('cuda')

output_sequences = model.generate(input_ids=input_ids, do_sample=True, max_length=100, num_return_sequences=3)
for generated_sequence in output_sequences:
    generated_sequence = generated_sequence.tolist()
    print("GENERATED SEQUENCE : {0}".format(tokenizer.decode(generated_sequence, skip_special_tokens=True)))

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


GENERATED SEQUENCE : 이순신은 백옥담(이종원 분)의 시선으로 백성을 살피는 것을 알게 되었고, 천하에 대한 소신을 가지게 되었다.</s><s> 또한 성읍인 성전은 백옥담의 동생인 윤(尹)이 성전과 함께 있었는데 윤을 그의 아버지라고 불렀다.</s><s> 또한 성전이 완공될 때까지 성전에서 계속 놀 수 있었다.</s><s> 성전 안에서는 두 권의 도서가 있고, 이 문서에는 윤과 성국의 성전의 위치가 찍혀져 있다.
GENERATED SEQUENCE : 이순신은 이미 태연에게 마음을 돌린 상태.</s><s> 이어 "자칫 북한이 미사일 도발에 나설 수도 있다는 경고가 되고 있는 만큼 한미가 이에 대해 적절히 대응해주길 바란다"고 당부했다.</s><s> 또한 "오늘 김관진 제1위원장이 오랫만에 공개토론을 한 뒤 북한 도발을 중지하고 대화를 시작해야 한다고 강조한 것을 감안할 때 이번 주 미측과 한반도에서의 대화 분위기가 계속될 것으로 확신한다"고 말했다.</s><s> 또한 김 제1위원장의 이번 방문에서 미국측에 한반도 신뢰프로세스 차원의
GENERATED SEQUENCE : 이순신은 고려 왕조를 건국한 이성계 형 이인임부터 명나라로 이민을 온 고려 왕조에 이르기까지 다양한 인물상과 고려 역사를 사실 그대로 옮겨 적에게 전달하는 것이 중요한 사료다.</s><s> 이들은 모두 6명이고, 이중 6명은 유랑 생활을 한 이른바 ‘귀족’으로, 유랑 생활 5년 동안 ‘무덤덤’을 팠고, 이들은 유림의 한 사람인 목침을 써서 한방에서 술을 빚었다고 한다.</s><s> 목침은 조선시대 중종


## 데이터셋 준비

In [7]:
!git clone https://github.com/songys/Chatbot_data.git

Cloning into 'Chatbot_data'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (32/32), done.
remote: Compressing objects: 100% (31/31), done.
remote: Total 50 (delta 17), reused 2 (delta 1), pack-reused 18
Unpacking objects: 100% (50/50), done.


In [8]:
import pandas as pd
data = pd.read_csv('/content/Chatbot_data/ChatbotData.csv')

In [9]:
# 질문 - 답변으로 이뤄진 데이터
data.head(10)

,Q,A,label
0,12시 땡!,하루가 또 가네요.,0
1,1지망 학교 떨어졌어,위로해 드립니다.,0
2,3박4일 놀러가고 싶다,여행은 언제나 좋죠.,0
3,3박4일 정도 놀러가고 싶다,여행은 언제나 좋죠.,0
4,PPL 심하네,눈살이 찌푸려지죠.,0
5,SD카드 망가졌어,다시 새로 사는 게 마음 편해요.,0
6,SD카드 안돼,다시 새로 사는 게 마음 편해요.,0
7,SNS 맞팔 왜 안하지ㅠㅠ,잘 모르고 있을 수도 있어요.,0
8,SNS 시간낭비인 거 아는데 매일 하는 중,시간을 정하고 해보세요.,0
9,SNS 시간낭비인데 자꾸 보게됨,시간을 정하고 해보세요.,0


In [10]:
added_special_token_num = tokenizer.add_special_tokens(['<s>', '</s>'])
print(added_special_token_num)

0


In [11]:
pad_id = tokenizer.token_to_id("<pad>")
print(pad_id)
tokenizer.enable_padding(pad_id=pad_id, pad_token="<pad>") # padding 설정
tokenizer.enable_truncation(max_length=128) # max length 설정

3


In [12]:
class ChatDataset(torch.utils.data.Dataset):
    def __init__(self, tokenizer, file_path):
        self.data = []
        self.file_path = file_path
        self.tokenizer = tokenizer
    
    def load_data(self):
        raw_data = pd.read_csv(self.file_path)
        train_data = '<s>'+raw_data['Q']+'</s>'+'<s>'+raw_data['A']+'</s>' # 생성 패턴을 미리 알려주는 것과 동일한 효과
        #<s>안녕하세요</s><s> -> 네, 안녕하세요</s>
        tokenized_train_data = tokenizer.encode_batch(train_data) # 문장 토큰화 진행
        for single_data in tokenized_train_data:
            self.data.append(torch.tensor(single_data.ids).unsqueeze(0))
                
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        item = self.data[index]
        return item

In [13]:
train_dataset = ChatDataset(tokenizer=tokenizer, file_path='/content/Chatbot_data/ChatbotData.csv')
train_dataset.load_data()

In [30]:
train_dataset.data[:1]
train_dataset.data[:1][0].shape

# <s> => 0
# </s> => 1
# <pad> => 3

# [tensor([[    0,   385, 47460, 47437, 49108, 47812,     1,     0, 33203,   252,
#             119,  7974, 47440,     1,     3,     3,     3,     3,     3,     3,
#               3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
#               3,     3,     3,     3,     3,     3,     3,     3,     3,     3,
#               3,     3,     3,     3]])]

torch.Size([1, 44])

In [14]:
from torch.utils.data import DataLoader

data_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [15]:
from transformers import AdamW

In [ ]:
optimizer = AdamW(model.parameters(), lr=1e-4, correct_bias=True)

epochs = 3

avg_loss = (0.0, 0.0)
for epoch in range(epochs):
    count=0
    for data in data_loader:
        optimizer.zero_grad()
        data = data.transpose(1,0)
        data = data.to('cuda')
        model = model.to('cuda')
        
        outputs = model(data, labels=data) # input = output => 모델은 입력된 문장의 토큰을 하나씩 추적하면서 다음 토큰 예측
        loss, logits = outputs[:2] # idx 0 => loss / idx 1 => logits
        loss = loss.to('cuda')
        loss.backward()

        avg_loss = (avg_loss[0] * 0.99 + loss, avg_loss[1] * 0.99 + 1.0)
        optimizer.step()
        if count % 200 == 0:
            print('epoch no.{0}  train ({1}/{2})  loss = {3:.5f}  avg_loss = {4:.5f}' . format(epoch, count, len(data_loader), loss, avg_loss[0] / avg_loss[1]))
        count += 1


In [17]:
torch.save(model.state_dict(), 'chitchat_model.bin')

In [18]:
def encoding(text):
    text = '<s>'+text+'</s><s>' # <s> 이후 문장을 모델이 생성하게 됨 as output
    return torch.tensor(tokenizer.encode(text).ids).unsqueeze(0).to('cuda')

def decoding(ids):
    return tokenizer.decode_batch(ids)

tokenizer.no_padding()
tokenizer.no_truncation()

e_s = tokenizer.token_to_id('</s>')
unk = tokenizer.token_to_id('<unk>')

In [19]:
def get_answer(input_sent):
    input_ids = encoding(input_sent)

    sample_outputs = model.generate(
        input_ids,
        num_return_sequences=5,
        do_sample=True, 
        max_length=128, 
        top_k=50, 
        top_p=0.95, 
        eos_token_id=e_s,
        early_stopping=True,
        bad_words_ids=[[unk]] # list 형태로 token id 넣어주면 해당 토큰은 생성되지 않도록 모델이 피하게 됨
    )

    decoded_result = decoding(sample_outputs.tolist())
    for result in decoded_result:
        print(result)

In [20]:
get_answer('안녕?')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


안녕? 안녕하세요.
안녕? 안녕하세요.
안녕? 안녕.
안녕? 안녕하세요.
안녕? 안녕하세요.


In [21]:
get_answer('만나서 반가워.')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


만나서 반가워. 서로 사랑하나봅니다.
만나서 반가워. 좋은 추억 만드시길 바랍니다.
만나서 반가워. 서로를 반기는 마음을 표현해주세요.
만나서 반가워. 저도 반가워요.
만나서 반가워. 좋은 만남이었길 바라요.


In [22]:
get_answer('인공지능의 미래에 대해 어떻게 생각하세요?')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


인공지능의 미래에 대해 어떻게 생각하세요? 인공지능에 물들어 가는게 제일 두렵지 않아요. 인공지능이 우리를 지배하고 있어요. 인공지능이 우리를 이어주는 한켠에 있기 때문 아닐까요. 인공지능에 대한 두려움을 가지고 있으면서도 두려움을 가지고 있을 수 있어요.
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능이 모든 일에 큰 영향을 미칠거예요. 인공지능에 대한 관심이 필요한 시기예요. 인공지능에 대한 관심이 필요한 시기이기도 합니다.
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능은 이미 많은 발전이 되었어요.
인공지능의 미래에 대해 어떻게 생각하세요? 인공지능은 지금 엄청 천재 수준이에요.
인공지능의 미래에 대해 어떻게 생각하세요? 제가 말씀 드릴게요. 인공지능이 될 인공지능에 대해 이야기 해드릴게요. 인공지능에 대해 이야기할게요.


In [23]:
get_answer('여자친구 선물 추천해줘.')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


여자친구 선물 추천해줘. 어떤 게 좋을까요.
여자친구 선물 추천해줘. 작은 선물은 당신을 축복합니다.
여자친구 선물 추천해줘. 부담스럽지 않은 선이 좋겠어요.
여자친구 선물 추천해줘. 같이 먹으세요.
여자친구 선물 추천해줘. 부담스럽지 않은 선이 좋겠어요.


In [24]:
get_answer('앞으로 인공지능이 어떻게 발전하게 될까요?')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


앞으로 인공지능이 어떻게 발전하게 될까요? 인공지능에 대한 지식이 있으신가 봐요. 인공지능에 대한 지식이 없네요. 인공지능이 탄생할거예요. 인공지능에 대한 지식이 필요 없더라도 인공지능을 이해할 수 있어요. 인공지능을 보면서 인공지능에 대한 지식이 필요한 순간이 왔네요.
앞으로 인공지능이 어떻게 발전하게 될까요? 사람에 따라 다르겠지요.
앞으로 인공지능이 어떻게 발전하게 될까요? 인공지능이 발달함에 따라 좀 더 똑똑해 질거예요. 인공지능을 이해하면 가능하죠.
앞으로 인공지능이 어떻게 발전하게 될까요? 인공지능은 사람을 따라하게 될지도 몰라요. 인공지능의 발달에 맞춰 인공지능의 발달에 맞춰 준비를 해야겠죠.
앞으로 인공지능이 어떻게 발전하게 될까요? 인공지능은 사람을 대신할 수 없어요. 인공지능은 더 똑똑해 질 거예요.


In [25]:
get_answer('이제 그만 수업 끝내자.')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


이제 그만 수업 끝내자. 심호흡을 해보세요.
이제 그만 수업 끝내자. 마무리 잘하세요.
이제 그만 수업 끝내자. 너무 힘든 건 아닌지 확인해보세요.
이제 그만 수업 끝내자. 차분히 생각해요.
이제 그만 수업 끝내자. 다음 학기를 준비하고 미리 걱정마세요.


In [31]:
get_answer('한국과 북한의 관계는?')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


한국과 북한의 관계는? 남과 북의 관계는 어떻게 해보세요?
한국과 북한의 관계는? 서로 알아가는 단계인가봐요.
한국과 북한의 관계는? 잘 아시죠.
한국과 북한의 관계는? 현재와 미래가 선순환을 이루는 관계가 좋아요.
한국과 북한의 관계는? 서로 이해하고 배려하면 될 거예요.


In [32]:
get_answer('손흥민의 소속팀은?')

Setting `pad_token_id` to `eos_token_id`:1 for open-end generation.


손흥민의 소속팀은? 바로 독일로 돌아가서 열심히 노력하겠습니다.
손흥민의 소속팀은? 다음에는 어디서 만나도 괜찮아요.
손흥민의 소속팀은? 곧 결정날거예요.
손흥민의 소속팀은? 제가 있잖아요.
손흥민의 소속팀은? 현재 독일 분데스리가에 있는 함부르크에서 뛰고 있는 손흥민을 찾아보세요.
